In [12]:
from podcast_ad_skipper.google_cloud import *
from podcast_ad_skipper.params import *
from podcast_ad_skipper.data_preparation import *
import pandas as pd
import json
from podcast_ad_skipper.main import get_processed_training_data

In [13]:
storage_client = auth_gc_storage()


Running in local environment.
Authenticated successfully with GCS! ✅


In [14]:
bucket_name = BUCKET_NAME
prefixes=GCP_PREFIXES[0]
chunk_size=5


In [15]:
file_list = retrieve_files_in_folder(storage_client, bucket_name, prefixes)

In [16]:
chunks_list = make_chunks(file_list, chunk_size)

In [17]:
%%time

bq_client = auth_gc_bigquery()

for chunk in chunks_list[:60]:

    features = get_features_model(chunk)

    table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.test9"

    rows_to_insert = [
        {
            "spectrogram": json.dumps(features[0][i].tolist()),
            "labels": features[1][i],
            "seconds": features[2][i],
            "duration": features[3][i],
            "podcast_names": features[4][i],
        } for i in range(len(features[0]))
    ]
    insert_data_to_bq(rows_to_insert, bq_client, table_id, chunk_size)
    # Time out error again, with both [:1] and [:10]
    # Move spectrogram code to model pipeline

Running in local environment.
Authenticated successfully with BigQuery! ✅
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added.
Batch of 5 rows has been added

In [18]:
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.test9"

In [19]:
bq_client = auth_gc_bigquery()

Running in local environment.
Authenticated successfully with BigQuery! ✅


In [20]:
output= get_output_query_bigquery(bq_client, table_id)


Query returned 500 results


In [21]:
processed_output = get_bq_processed_data(output)

In [24]:
processed_output[0][200].shape

(128, 216)

In [25]:
nans_count = 0
spec_column = processed_output[0]

def check_for_nans(spec_column, nans_count):
    for idx, array in enumerate(spec_column):
        if np.isnan(array).any():
                # print(f"NaNs found in array at index {idx}")
            nans_count += 1
    print(nans_count)

In [26]:
check_for_nans(spec_column, nans_count)

0
